In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os

import scipy.stats as stats

from sklearn.preprocessing import StandardScaler, Normalizer, LabelEncoder, PolynomialFeatures
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold, train_test_split, cross_val_score, cross_val_predict, GridSearchCV, RandomizedSearchCV

from sklearn.cluster import KMeans
from sklearn.metrics.cluster import silhouette_score

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC  
from sklearn import metrics
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error, accuracy_score

#visualizing results
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

In [ ]:
path = 'C:/Users/Schindler/Documents/Schindler_Lab/ML projects/TILES/Data/EMA surveys -MOSAIC.csv'

In [ ]:
data = pd.read_csv(path)
data = pd.DataFrame(data = data)
print('Original MOSAIC EMA shape:\n', data.shape, '\n')
print('Original MOSAIC EMA survey type counts:\n', data['survey_type'].value_counts(), '\n')
print('Original MOSAIC EMA missing value couts:\n', data.isnull().sum(), '\n')
print('Original MOSAIC EMA data types:\n', data.info(), '\n')
data.head()

In [ ]:
#select survey type
data_health_orig = data[data['survey_type'] == 'health']
print('Original MOSAIC health survey shape:\n', data_health_orig.shape, '\n')
#set index
data_health_orig.set_index(['uid', 'timestamp', 'survey_type'], inplace=True)

#find how many missing
print('Original MOSAIC health survey data missing value counts:\n', data_health_orig.isnull().sum(), '\n')
#drop columns with high missing
data_health = data_health_orig.dropna(axis=1, thresh=300)
print('Data shape after dropping columns:\n', data_health.shape, '\n')
#find out if still missing
print('Still missing:\n', data_health.isnull().sum(), '\n')
#drop anything with a na
data_health_nonull = data_health.dropna()
#confirm no more missing
print('Missing check:\n', data_health_nonull.isnull().sum(), '\n')

print('No null data_job shape:\n', data_health_nonull.shape, '\n')

print("Fraction of data kept:\n",float(data_health_nonull.shape[0])/data_health.shape[0])

data_health_nonull.head()

In [ ]:
sns.pairplot(data_health_nonull, kind = 'reg')